# Práctica 4 #  
Grupo: Tralalelo Tralala  
Integrantes: Yuriy Chaban Markevych, Pablo García Fernández, Alejandro Rodríguez Giner y Alonso Campillo Martínez  

# Introducción  
Aquí explicamos el trabajo y la motivación por hacerlo

# Dataset  
Explicamos el dataset elegido, las columnas que vamos a usar y hacemos la limpieza de los datos (previo análisis). Vamos a tratar todos los campos del dataframe como strings. Si en algún futuro necesitamos tratar los datos como números, simplemente los convertiremos.

In [ ]:
import sklearn as skl
import pandas as pd
import numpy as np

_prestamos = pd.read_csv('dataset.csv')


Ahora vamos a mostrar los 4 primeros y los 4 últimos para asegurar que funciona el dataset a la perfección.

In [ ]:
display(_prestamos.head(n=4))
display(_prestamos.tail(n=4))
print(_prestamos.columns.tolist())

Limpiamos el dataframe para que no tengamos problemas con las claves y las columnas en el futuro, concretamente quitando los espacios en blanco 
y quitando el loan_id (ya que es un dato innecesario para la predicción del modelo).

In [ ]:
_prestamos.columns = _prestamos.columns.str.strip()
_prestamos = _prestamos.applymap(lambda x: x.strip() if isinstance(x, str) else x)
display(_prestamos.head(n=1))

columnas= ["no_of_dependents", "education", "self_employed", "income_annum", "loan_amount", "loan_term", "cibil_score", "residential_assets_value", "commercial_assets_value","luxury_assets_value","bank_asset_value","loan_status"] 
_prestamos= _prestamos[columnas]

display(_prestamos)

Aplicamos ahora One Hot Encoder a los atributos categóricos. Usaremos el método unique para identificar los valores únicos de esa columna, de manera que podamos detectar si hay algun valor erróneo. Finalmente, aplicamos OneHotEncoder para convertir la columna en valores binarios.

In [ ]:
self_employed_data= _prestamos['self_employed'].unique() 
print(self_employed_data)

education_data= _prestamos['education'].unique() 
print(education_data)

loan_status_data= _prestamos['loan_status'].unique() 
print(loan_status_data)

columnas_categoricas = ['self_employed', 'education', 'loan_status']

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output=False)

categorical_encoded = encoder.fit_transform(_prestamos[columnas_categoricas])

categorical_encoded_df = pd.DataFrame(
    categorical_encoded,
    columns=encoder.get_feature_names_out(columnas_categoricas)
)

_prestamos = pd.concat(
    [_prestamos.drop(columns=columnas_categoricas), categorical_encoded_df],
    axis=1
)

display(_prestamos)


Ahora miramos los atributos numericos. A todas las columnas les aplicaremos la conversión a numérico para evitar que queden representados en otro tipo de datos (por ejemplo, String). Previo a ello miraremos los valores numéricos de cada columna con el fin de observar posibles valores extraños. Además, en el caso de los activos, cantidad solicitada y plazos revisaremos el valor mínimo, pues no debe ser negativo, y en el caso del CIBIL el valor máximo y mínimo, pues es una puntuación que debe estar entre 300 y 900.

In [ ]:
def no_negativos(x):
    if x < 0:
        return 0
    else:
        return x

dependents_data= _prestamos['no_of_dependents'].unique() 
print(dependents_data)
_prestamos['no_of_dependents'] = pd.to_numeric(_prestamos['no_of_dependents'], errors='coerce')

annum_data= _prestamos['income_annum'].unique() 
print(annum_data)
_prestamos['income_annum'] = pd.to_numeric(_prestamos['income_annum'], errors='coerce')

loan_amount_data = _prestamos['loan_amount'].unique()
print(loan_amount_data)
_prestamos['loan_amount'] = pd.to_numeric(_prestamos['loan_amount'], errors='coerce')
print(_prestamos['loan_amount'].min())

loan_term_data = _prestamos['loan_term'].unique()
print(loan_term_data)
_prestamos['loan_term'] = pd.to_numeric(_prestamos['loan_term'], errors='coerce')
print(_prestamos['loan_term'].min())

_prestamos['cibil_score'] = pd.to_numeric(_prestamos['cibil_score'], errors='coerce')
cibil_score_data = _prestamos['cibil_score'].unique()
print(cibil_score_data) 
print(_prestamos['cibil_score'].min(), _prestamos['cibil_score'].max()) 

residential_assets_value_data = _prestamos['residential_assets_value'].unique()
print(residential_assets_value_data)
_prestamos['residential_assets_value'] = pd.to_numeric(_prestamos['residential_assets_value'], errors='coerce')
print(_prestamos['residential_assets_value'].min())
#Como tiene valores negativos los convertimos a 0, ya que no pueden ser negativos
_prestamos['residential_assets_value'] = _prestamos['residential_assets_value'].apply(no_negativos)
print(_prestamos['residential_assets_value'].min())


commercial_assets_value_data = _prestamos['commercial_assets_value'].unique()
print(commercial_assets_value_data) 
_prestamos['commercial_assets_value'] = pd.to_numeric(_prestamos['commercial_assets_value'], errors='coerce')
print(_prestamos['commercial_assets_value'].min())


luxury_assets_value_data = _prestamos['luxury_assets_value'].unique()
print(luxury_assets_value_data)
_prestamos['luxury_assets_value'] = pd.to_numeric(_prestamos['luxury_assets_value'], errors='coerce')
print(_prestamos['luxury_assets_value'].min())

bank_asset_value_data = _prestamos['bank_asset_value'].unique()
print(bank_asset_value_data)
_prestamos['bank_asset_value'] = pd.to_numeric(_prestamos['bank_asset_value'], errors='coerce')
print(_prestamos['bank_asset_value'].min())

Por último, para facilitar el desarrollo de los distintos modelos, vamos a renombrar las columnas y vamos a guardar el resultado en un archivo csv, de manera que para cada nueva evaluación se cargue el fichero ya limpio.

In [ ]:
columnas = {'no_of_dependents':'Personas a cargo',
            'income_annum': 'Ingreso Anual',
            'loan_amount': 'Cantidad del prestamo',
            'loan_term' : 'Plazos',
            'cibil_score': 'Puntos CIBIL',
            'residential_assets_value': 'Activos residenciales',
            'commercial_assets_value': 'Activos comerciales',
            'luxury_assets_value': 'Activos de lujo',
            'bank_asset_value': 'Activos bancarios',
            'self_employed_Yes': 'Autonomo',
            'education_Not Graduate': 'No graduado',
            'loan_status_Rejected': 'Prestamo rechazado'
}

_prestamos.rename(columns=columnas, inplace=True)
_prestamos.to_csv('dataset_clean.csv', index=False)

# Evaluación  
Aquí va la predicción como tal (añadir gráficas si es posible)

### KNN  
desarrollo

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('dataset_clean.csv')

x = data.drop(columns=['Prestamo rechazado'])
y = data['Prestamo rechazado']

x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#Se han hecho múltiples pruebas de k sin fijar random_state y el mejor score suele estar sobre 70
model = knc(n_neighbors=70, weights='distance')

model.fit(x_train, y_train)
prediction = model.predict(x_test)
print(f"Predicción: {prediction}")
print(f"Test: {y_test}")

score = model.score(x_test, y_test)
print(f"Score: {score}")

# Conclusiones  
Conclusiones sacadas del estudio